In [1]:
from scipy.optimize import brute
import numpy as np
from datetime import datetime,timedelta
import pandas as pd

In [2]:
# Load solver
import ray
import LearnerYabox_v2a as Learner 

2020-05-18 15:50:35,154	WARNING resource_spec.py:163 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_memory` when calling ray.init() or ray start.
2020-05-18 15:50:35,156	INFO resource_spec.py:205 -- Starting Ray with 120.7 GiB memory available for workers and up to 18.63 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


In [3]:
import ray

In [5]:
# Initial parameters
dfparam = pd.read_csv("data/param.csv")
dfparam

,DRS,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,DRS 01 - Grande São Paulo,2020-03-22,150,900000.0,0.0001,0.0001,0.0001,0.0001,80.0000,1500,0.1,0.6,0.1
1,DRS 02 - Araçatuba,2020-04-01,150,500.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
2,DRS 03 - Araraquara,2020-04-01,150,2000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
3,DRS 04 - Baixada Santista,2020-04-01,150,8000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
4,DRS 05 - Barretos,2020-04-01,150,2000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
5,DRS 06 - Bauru,2020-04-01,150,10000.0,0.0001,0.0001,4.0000,0.0001,0.0001,0,0.1,0.4,0.1
6,DRS 07 - Campinas,2020-04-01,150,20000.0,0.0001,0.0001,40.0000,0.0001,0.0001,0,0.1,0.4,0.1
7,DRS 08 - Franca,2020-04-01,150,1000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
8,DRS 09 - Marília,2020-04-01,150,5000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
9,DRS 10 - Piracicaba,2020-04-01,150,10000.0,0.0001,0.0001,0.0001,0.0001,1.0000,0,0.1,0.4,0.1


In [6]:
df = pd.read_csv("data/dados_total_estados.csv")
df

In [7]:
#@ray.remote
def opt(state,e0,a0,r0,d0,date,version):
    rranges = [slice(1e5,2e6,1e5),slice(-2,2,4),slice(0,1000,1000),slice(0.1,0.6,0.1),slice(0.1,0.2,0.05)]
    optimal = brute(fobjective,        
        ranges=rranges,
        args=(state,e0,a0,r0,d0,date,version), full_output=True, disp=True, finish=None)
    
    return optimal

In [8]:
from scipy.integrate import odeint

def fobjective(point,state,e0,a0,r0,d0,date,version):
    results=[]
    s0, deltaDate, i0, weigthCases, weigthRecov = point
    
    startNCases = 20
    ratio = 0.1
    version = 1
    cleanRecovered = False
    
    Date = datetime.strptime(date, "%Y-%m-%d")
    end_date = Date + timedelta(days=+int(deltaDate))
    dateStr=end_date.strftime("%Y-%m-%d")
    
    predict_range = 120
    
    learner = Learner.Learner.remote(state,dateStr, 
                                     predict_range,s0, e0, a0, i0, r0, d0, 
                                     startNCases, ratio, wcases, wrec,
                                     cleanRecovered, version, savedata=False)
    
    results.append(learner.train.remote())
    results = ray.get(results)
    return results

In [10]:
states=['SP','ES','MG','RJ','PE','CE','AM']
allStates=False
version="10"

optimal=[]
if allStates:
    for state in states:
    #for districtRegion in DRS:
        query = dfparam.query('state == "{}"'.format(state)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        startdate,predictionrange,s0,e0,a0,i0,r0,d0,START,ratio,wcases,wrec = parameters
        optimal = []
        optimal.append(opt(state,e0,a0,r0,d0,startdate,version)) 
        print("State = {}\n".format(state)
        print("S0 = {}\n".format(optimal[i][0][0]))
        print("Delta Date Days = {}\n".format(optimal[i][0][1]))   
        print("I0 = {}\n".format(optimal[i][0][2]))
        print("wcases = {}\n".format(optimal[i][0][3]))
        print("wrec = {}\n".format(optimal[i][0][4]))
        print("Function Minimum = {}\n".format(optimal[i][1]))
        j = query['index'].values[0]
        dfparam.at[j, "s0"] = optimal[i][0][0]
        dfparam.at[j, "i0"] = optimal[i][0][2]
        dfparam.at[j, "WCASES"] = optimal[i][0][3]
        dfparam.at[j, "WREC"] = optimal[i][0][4]
        
else:
    state = "ES" 
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    startdate,predictionrange,s0,e0,a0,i0,r0,d0,START,ratio,wcases,wrec = parameters
    optimal = []
    optimal.append(opt(state,e0,a0,r0,d0,startdate,version))
    
    print("state = {}\n".format(state))
    print("S0 = {}\n".format(optimal[0][0][0]))
    print("Delta Date Days = {}\n".format(optimal[0][0][1]))   
    print("I0 = {}\n".format(optimal[0][0][2])) 
    print("wcases = {}\n".format(optimal[i][0][3]))
    print("wrec = {}\n".format(optimal[i][0][4]))
    print("Function Minimum = {}\n".format(optimal[0][1]))
    
    j = query['index'].values[0]
    dfparam.at[j, "s0"] = optimal[0][0][0]
    dfparam.at[j, "i0"] = optimal[0][0][2]
    dfparam.at[j, "WCASES"] = optimal[i][0][3]
    dfparam.at[j, "WREC"] = optimal[i][0][4]

RayTaskError: [36mray_Learner:train()[39m (pid=22951, host=jedha)
  File "/home/ats4i/Desktop/corona/dataAndModelsCovid19/regionsSP/Learner.py", line 152, in train
    end=datetime.strptime(new_index[len(new_index)-2],'%Y-%m-%d'))
  File "/home/ats4i/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py", line 5192, in __setattr__
    return object.__setattr__(self, name, value)
  File "pandas/_libs/properties.pyx", line 67, in pandas._libs.properties.AxisProperty.__set__
  File "/home/ats4i/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py", line 690, in _set_axis
    self._data.set_axis(axis, labels)
  File "/home/ats4i/anaconda3/lib/python3.7/site-packages/pandas/core/internals/managers.py", line 183, in set_axis
    "values have {new} elements".format(old=old_len, new=new_len)
ValueError: Length mismatch: Expected axis has 60 elements, new values have 59 elements


In [11]:
ray.shutdown()

In [13]:
dfparam


,DRS,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,DRS 01 - Grande São Paulo,2020-03-22,150,900000.0,0.0001,0.0001,0.0001,0.0001,80.0000,1500,0.1,0.6,0.1
1,DRS 02 - Araçatuba,2020-04-01,150,500.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
2,DRS 03 - Araraquara,2020-04-01,150,2000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
3,DRS 04 - Baixada Santista,2020-04-01,150,8000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
4,DRS 05 - Barretos,2020-04-01,150,2000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
5,DRS 06 - Bauru,2020-04-01,150,10000.0,0.0001,0.0001,4.0000,0.0001,0.0001,0,0.1,0.4,0.1
6,DRS 07 - Campinas,2020-04-01,150,20000.0,0.0001,0.0001,40.0000,0.0001,0.0001,0,0.1,0.4,0.1
7,DRS 08 - Franca,2020-04-01,150,1000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
8,DRS 09 - Marília,2020-04-01,150,5000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
9,DRS 10 - Piracicaba,2020-04-01,150,10000.0,0.0001,0.0001,0.0001,0.0001,1.0000,0,0.1,0.4,0.1


In [ ]:
dfparam.to_csv("data/param_optimized.csv", sep=",", index=False)